In [28]:
import pandas as pd
import numpy as np
import traceback

from esa_snappy import ProductIO, HashMap
from esa_snappy import GeoPos
from esa_snappy import PixelPos
from esa_snappy import GPF
import rasterio
from rasterio.transform import from_origin

# from osgeo import gdal, osr

from glob import glob
from tqdm import tqdm
import os
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

# Suppress specific warning messages
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="org.esa.snap.core.dataio.dimap.DimapProductReader")


In [ ]:
df_ksa = pd.read_csv("/data/raw/processed/cloned_points.csv")
df_ksa['idprov'] = df_ksa.idsegmen.astype('str').str[:2]
df_ksa['idkab'] = df_ksa.idsegmen.astype('str').str[:4]


In [7]:
df_ksa = df_ksa.loc[df_ksa.idkab == '3212'] 
df_ksa.head()

,iterx,itery,lat,long,index,idsegmen,strati,idsubsegmen,EASTING,NORTHING,100kmSQ_ID,GZD,MGRS,idprov,idkab
1890900,1,1,-6.410425,107.946818,1,321201001,S1,A1,800000mE,9200000mN,ZT,48M,48MZT,32,3212
1890901,1,2,-6.410604,107.946818,2,321201001,S1,A1,800000mE,9200000mN,ZT,48M,48MZT,32,3212
1890902,1,3,-6.410782,107.946818,3,321201001,S1,A1,800000mE,9200000mN,ZT,48M,48MZT,32,3212
1890903,1,4,-6.410961,107.946818,4,321201001,S1,A1,800000mE,9200000mN,ZT,48M,48MZT,32,3212
1890904,1,5,-6.411139,107.946818,5,321201001,S1,A1,800000mE,9200000mN,ZT,48M,48MZT,32,3212


In [8]:
df_ksa.MGRS.unique()

array(['48MZT', '49MAN', '49MBN', '49MAP', '49MBP', '48MZU'], dtype=object)

In [25]:
# Load the .dim file
path_to_sentinel_data = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/48MZT/20230501_20230512.dim"
product_subset = ProductIO.readProduct(path_to_sentinel_data)
# Define parameters for the GeoTIFF export
params = HashMap()
params

java.util.HashMap(objectRef=0xae47098)

In [42]:
# Load the .dim file
path_to_sentinel_data = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/48MZT/20230501_20230512.dim"
product = ProductIO.readProduct(path_to_sentinel_data)

# Extract bands and their data
bands = list(product.getBandNames())
band_arrays = []

for band_name in bands:
    band = product.getBand(band_name)
    width = band.getRasterWidth()
    height = band.getRasterHeight()
    band_data = np.zeros((height, width), np.float32)
    band.readPixels(0, 0, width, height, band_data)
    band_arrays.append(band_data)

# Extract geocoding information
gc = product.getSceneGeoCoding()

# Get the raster size
width = band_arrays[0].shape[1]
height = band_arrays[0].shape[0]

# We will use the upper-left corner pixel for geocoding
# Attempting to use an alternative approach to retrieve the top-left position
try:
    # Create a more general approach to obtain the top-left corner position
    geo_pos = gc.getGeoPos(0, 0, None)  # Adjust if necessary
    top_left_lon = geo_pos.lon
    top_left_lat = geo_pos.lat
except Exception as e:
    print("Error retrieving geographical position:", e)
    top_left_lon = 0  # Set a default value or handle accordingly
    top_left_lat = 0  # Set a default value or handle accordingly

# Define pixel sizes (ensure these are set correctly based on your data)
pixel_size_x = 20  # Replace with actual pixel size
pixel_size_y = 20  # Replace with actual pixel size

# Create a transform for the GeoTIFF
transform = from_origin(top_left_lon, top_left_lat, pixel_size_x, pixel_size_y)

# Define the output path for the GeoTIFF
output_path = "sentinel_data_3857.tif"

# Export as GeoTIFF using rasterio
with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=len(band_arrays),
        dtype=band_arrays[0].dtype,
        crs='EPSG:3857',  # Change this if you know the specific CRS
        transform=transform) as dst:

    for i, band_data in enumerate(band_arrays, 1):
        dst.write(band_data, i)

print(f"GeoTIFF saved at: {output_path}")

Error retrieving geographical position: no matching Java method overloads found
GeoTIFF saved at: sentinel_data_3857.tif
